In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 22.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [3]:
import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from mxnet.gluon.data import DataLoader
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
import numpy as np
import os

In [4]:
from mxnet import test_utils

try:
    # Try to run a simple MXNet operation on GPU
    test_utils.list_gpus()
    print("GPU is available and configured.")
except mx.MXNetError:
    print("No GPU is available or configured.")

GPU is available and configured.


In [5]:
ctx = mx.cpu()

In [6]:
data_dir = "/content/drive/MyDrive/deep_learning_project/face_expression_dataset"

In [7]:
def load_fer2013_dataset(data_dir, batch_size=64):
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'test')
    test_dir = os.path.join(data_dir, 'test')

    # Define data transformations (resize, normalize, etc.)
    transform = transforms.Compose([
        transforms.Resize(48),  # Resize images to a fixed size (adjust as needed)
        transforms.ToTensor(),   # Convert to tensor and normalize to [0, 1]
        transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
    ])

    # Load datasets and apply transformations
    train_data = DataLoader(
        ImageFolderDataset(train_dir).transform_first(transform),
        batch_size=batch_size, shuffle=True, num_workers=4
    )

    val_data = DataLoader(
        ImageFolderDataset(val_dir).transform_first(transform),
        batch_size=batch_size, num_workers=4
    )

    test_data = DataLoader(
        ImageFolderDataset(test_dir).transform_first(transform),
        batch_size=batch_size, num_workers=4
    )

    return train_data, val_data, test_data

In [8]:
train_data, val_data, test_data = load_fer2013_dataset(data_dir)

In [9]:
class EmotionNet(nn.HybridBlock):
    def __init__(self, num_classes):
        super(EmotionNet, self).__init__()
        with self.name_scope():
            self.conv1 = nn.Conv2D(32, kernel_size=(3, 3))
            self.pool1 = nn.MaxPool2D(pool_size=(2, 2))
            self.conv2 = nn.Conv2D(64, kernel_size=(3, 3))
            self.pool2 = nn.MaxPool2D(pool_size=(2, 2))
            self.flatten = nn.Flatten()
            self.fc1 = nn.Dense(128, activation='relu')
            self.fc2 = nn.Dense(num_classes)

    def hybrid_forward(self, F, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
net = EmotionNet(num_classes=7)
net.initialize(mx.init.Xavier(), ctx=ctx)

In [10]:
loss_fn = mx.gluon.loss.SoftmaxCrossEntropyLoss()
optimizer = mx.gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.001})

In [14]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = mx.metric.Accuracy()

    for data, label in train_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)

        with mx.autograd.record():
            output = net(data)
            loss = loss_fn(output, label)
        loss.backward()
        optimizer.step(data.shape[0])

        train_loss += loss.mean().asscalar()
        train_acc.update(label, output)

    print(f"Epoch {epoch + 1}, Loss: {train_loss / len(train_data)}, Train Acc: {train_acc.get()[1]}")

Epoch 1, Loss: 1.7525503937877827, Train Acc: 0.3022519352568614
Epoch 2, Loss: 1.5144730272577769, Train Acc: 0.42423176167018534
Epoch 3, Loss: 1.350866722526835, Train Acc: 0.5010555946516537
Epoch 4, Loss: 1.1792063290503487, Train Acc: 0.5654468684025334
Epoch 5, Loss: 1.0176628409926571, Train Acc: 0.636406286652592
Epoch 6, Loss: 0.8161997781760657, Train Acc: 0.7158104621158808
Epoch 7, Loss: 0.6091211527141173, Train Acc: 0.7931034482758621
Epoch 8, Loss: 0.42850782321905023, Train Acc: 0.8628899835796388
Epoch 9, Loss: 0.2722714914759593, Train Acc: 0.9181327703495191
Epoch 10, Loss: 0.174024382411544, Train Acc: 0.9534365470326062


In [16]:
# Save the trained model
net.save_parameters('/content/drive/MyDrive/deep_learning_project/emotion_recognition_model.params')

In [17]:
# Load the saved model
net.load_parameters('/content/drive/MyDrive/deep_learning_project/emotion_recognition_model.params')
test_acc = mx.metric.Accuracy()
for data, label in test_data:
    data = data.as_in_context(ctx)
    label = label.as_in_context(ctx)
    output = net(data)
    test_acc.update(label, output)

print(f"Test Accuracy: {test_acc.get()[1]}")

Test Accuracy: 0.45500848896434637
